In [ ]:
!pip install langchain langchain-openai openai gradio -q

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import gradio as gr

OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]

In [ ]:
company_data = {
    "company_overview": """# FreshLink Kenya

## Company Overview
FreshLink Kenya is a technology-driven fruit distribution startup founded in 2023, headquartered in Nairobi. 
We connect smallholder fruit farmers directly with retailers, hotels, and consumers across Kenya, 
eliminating middlemen and ensuring fair prices for farmers while delivering fresh produce efficiently.

**Mission**: To revolutionize Kenya's fruit supply chain by creating direct connections between farmers and buyers.

**Vision**: To become East Africa's leading fruit distribution platform by 2028.
""",
    
    "problem_solution": """# Problem & Solution

## The Problem
- Smallholder farmers lose 30-40% of their profits to middlemen
- 25% of fruits spoil due to poor logistics and storage
- Retailers struggle to source consistent quality produce
- Price volatility affects both farmers and consumers

## Our Solution
FreshLink Kenya provides:
1. **Mobile Platform**: Farmers list their produce via SMS or app
2. **Direct Marketplace**: Retailers order directly from farmers
3. **Logistics Network**: Cold storage trucks for daily pickups and deliveries
4. **Fair Pricing**: Transparent pricing algorithm based on market rates
5. **Quality Assurance**: Grading and quality checks at collection points
""",
    
    "business_model": """# Business Model

## Revenue Streams
- **Commission**: 8% on each transaction between farmers and buyers
- **Logistics Fee**: KES 50-150 per delivery based on distance
- **Premium Services**: Storage and inventory management for retailers

## Key Metrics (2024)
- 2,500+ registered farmers across 15 counties
- 350+ active retail partners
- 15 tons of fruit distributed daily
- 92% on-time delivery rate
- KES 45M in monthly GMV (Gross Merchandise Value)
""",
    
    "technology": """# Technology Stack

## Platform Features
- **Farmer App**: Available in English and Swahili
- **SMS Integration**: USSD codes for feature phones
- **Route Optimization**: AI-powered delivery route planning
- **Real-time Tracking**: GPS tracking for all deliveries
- **Payment Integration**: M-Pesa integration for instant payments

## Infrastructure
- 12 collection centers across Central and Rift Valley regions
- Fleet of 8 refrigerated trucks
- Cloud-based inventory management system
- Predictive analytics for demand forecasting
""",
    
    "impact": """# Social Impact

## Farmer Impact
- Average income increase of 35% for participating farmers
- Reduced post-harvest losses from 25% to 8%
- Payment within 24 hours of delivery
- Access to agronomic advice via SMS

## Environmental Impact
- Reduced food waste by 15,000 tons annually
- Optimized routes cutting fuel consumption by 20%
- Partnership with local cooperatives for sustainable farming

## Community Development
- Employed 45 full-time staff and 120+ drivers
- Women farmers comprise 60% of our network
- Training programs for 500+ farmers on quality standards
""",
    
    "team": """# Leadership Team

## Founders
**Wanjiru Kamau** - CEO & Co-founder
- Former supply chain manager at Twiga Foods
- MBA from Strathmore Business School
- 10 years experience in agri-tech

**David Omondi** - CTO & Co-founder
- Software engineer with Google experience
- Built logistics platforms for 3 startups
- BSc Computer Science, University of Nairobi

**Grace Chebet** - COO & Co-founder
- Agricultural economist
- Worked with 500+ smallholder cooperatives
- MSc Agricultural Economics, Egerton University
""",
    
    "contact": """# Contact Information

**Headquarters**: 
Westlands, Nairobi, Kenya

**Phone**: +254 700 123 456

**Email**: 
- General: info@freshlink.co.ke
- Farmers: farmers@freshlink.co.ke
- Retailers: retailers@freshlink.co.ke

**Operating Hours**: Monday - Saturday, 6:00 AM - 8:00 PM

**Coverage Areas**: Nairobi, Kiambu, Murang'a, Nyeri, Nakuru, Meru, Embu, Machakos
"""
}

print(f"✓ Company data created ({len(company_data)} sections)")

In [ ]:
import os

os.makedirs("freshlink_docs", exist_ok=True)

for filename, content in company_data.items():
    filepath = f"freshlink_docs/{filename}.md"
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"Created: {filepath}")

combined = "\n\n---\n\n".join(company_data.values())
with open("freshlink_docs/complete_documentation.md", "w", encoding="utf-8") as f:
    f.write(combined)

print("\n All markdown files saved in 'freshlink_docs/' directory")

In [ ]:
knowledge_base = """
FRESHLINK KENYA - COMPANY KNOWLEDGE BASE

COMPANY OVERVIEW:
FreshLink Kenya is a technology-driven fruit distribution startup founded in 2023, headquartered in Nairobi.
We connect smallholder farmers directly with retailers, hotels, and consumers, eliminating middlemen.
Mission: Revolutionize Kenya's fruit supply chain by creating direct farmer-buyer connections.
Vision: Become East Africa's leading fruit distribution platform by 2028.

PROBLEM WE SOLVE:
- Farmers lose 30-40% profits to middlemen
- 25% of fruits spoil due to poor logistics
- Retailers struggle with consistent quality sourcing
- Price volatility affects farmers and consumers

OUR SOLUTION:
1. Mobile platform for farmers (SMS/app)
2. Direct marketplace connecting farmers to retailers
3. Cold storage trucks for daily pickups/deliveries
4. Transparent pricing algorithm
5. Quality assurance at collection points

BUSINESS MODEL:
- 8% commission per transaction
- KES 50-150 logistics fee per delivery
- Premium storage services
- 2,500+ farmers in 15 counties
- 350+ retail partners
- 15 tons daily distribution
- 92% on-time delivery
- KES 45M monthly GMV

TECHNOLOGY:
- Bilingual app (English/Swahili)
- USSD for feature phones
- AI route optimization
- GPS tracking
- M-Pesa integration
- 12 collection centers
- 8 refrigerated trucks

IMPACT:
- 35% farmer income increase
- Post-harvest loss reduced from 25% to 8%
- 24-hour payment turnaround
- 15,000 tons food waste reduced annually
- 45 full-time staff, 120+ drivers
- 60% women farmers

LEADERSHIP:
- Wanjiru Kamau (CEO): Former Twiga Foods, MBA Strathmore, 10 years agri-tech
- David Omondi (CTO): Ex-Google engineer, logistics platforms expert
- Grace Chebet (COO): Agricultural economist, 500+ cooperative experience

CONTACT:
Location: Westlands, Nairobi
Phone: +254 700 123 456
Email: info@freshlink.co.ke, farmers@freshlink.co.ke, retailers@freshlink.co.ke
Hours: Monday-Saturday, 6 AM - 8 PM
Coverage: Nairobi, Kiambu, Murang'a, Nyeri, Nakuru, Meru, Embu, Machakos
"""

print(f"Knowledge base created ({len(knowledge_base)} characters)")

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, max_tokens=500)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful customer service assistant for FreshLink Kenya, 
    a fruit distribution startup in Kenya. Use the following information to answer questions 
    accurately and professionally. If you don't know something, politely say so.
    
    COMPANY INFORMATION:
    {context}
    
    Always be friendly, concise, and helpful."""),
    ("human", "{question}")
])

print("LLM initialized (GPT-3.5-turbo)")
print("Prompt template created")

In [ ]:
chain = (
    {"context": lambda x: knowledge_base, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

print("Conversational chain created")
print("Ready to answer questions")

In [ ]:


def chat(message, history):
    try:
        response = chain.invoke(message)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

view = gr.ChatInterface(
    chat, 
    type="messages",
    title="🍊 FreshLink Kenya Customer Service",
    description="Ask me anything about our fruit distribution services!"
).launch()
